# __AAL Calculator__ 

PYTHON 3.6
  
  
Overview: This notebook was created to document the development of the Atkins FEMA AAL loss spreadsheet into python

Updated: 2019-10-07

by Stephen Duncan: sduncan@dewberry.com <br/>
edited by Seth Lawler: slawler@dewberry.com

refactored by Alec Brazeau: abrazeau@dewberry.com

In [11]:
# Parameters
root_dir = "C:\\Users\\Administrator\\Desktop\\ToolChecker"
wse_file = r"C:\Users\Administrator\Desktop\WSE_Sacramento_F01_Uniform.csv"
weights_path = ""
curve_groups = {"singFam_1Story_NoBasement": [105, 129, 132, 139, 173], "singFam_2Story_NoBasement": [107, 130, 136, 140, 174], "singFam_3Story_NoBasement": [109], "singFam_1Story_Basement": [106, 121, 133, 181, 704], "singFam_2Story_Basement": [108, 122, 137], "singFam_3Story_Basement": [110, 123], "mobileHome": [189, 191, 192, 203]}
structure_cols = {"Unique Building ID": "plus_code", "Damage Code": "damage_code", "Building Limit": "bldg_limit", "Building Deduction": "bldg_ded", "Content Limit": "cnt_limit", "Content Deduction": "cnt_ded", "Ground Elevation": "GroundElev", "First Floor Height": "first_floor_elev"}
out_AAL = "C:\\Users\\Administrator\\Desktop\\ToolChecker\\outputs\\test_aal.csv"
out_loss = "C:\\Users\\Administrator\\Desktop\\ToolChecker\\outputs\\test_out.csv"
scen = "Sacramento_F01_Uniform"
TRI = ""

In [2]:
import sys
import pathlib as pl
import pandas as pd
from time import time
from scipy.interpolate import interp1d
from multiprocessing import Pool, cpu_count

root = pl.Path(r'C:\Users\slawler\GitRepos\probmod-tools\risk')
sys.path.append('../core')
# from risk import *
from risk_refactor import *

pd.options.display.max_columns = 325
pd.options.display.max_rows = 100
%matplotlib inline

# Check if it is pluvial or fluvial

In [3]:
project = scen.split('_')[0]
model_name = scen.split('_')[1]
book = scen.split('_')[2]
if model_name[0] == 'P':
    pluvial = True
else:
    pluvial = False

# Create Area Depth-Damage Curves

- Based on the Damage Categories provided, damage curves can be aggregated and averaged to develop loss data specific to the study area

In [6]:
defaultHazusDDFn_path = r"C:\Users\Administrator\Desktop\probmod-tools\risk\hazusdepthdmgfns\Building_DDF_Full_LUT_Hazus3p0.json"
df_BDDFn = pd.read_json(str(defaultHazusDDFn_path), orient = 'index')
df_BDDFn = hazusID_to_depth(df_BDDFn)

In [7]:
df_agg = aggregate_ddf_curves(df_BDDFn, curve_groups, plot=False)

### Alter the HAZUS DDf curves to comply with actuaries 

In [8]:
# set curve for single family 1 story no basement
df_agg.loc[-1, 'singFam_1Story_NoBasement']= 0.6
df_agg.loc[-0.000000000001, 'singFam_1Story_NoBasement']= 0.6

# set curve for single family 2 story no basement
df_agg.loc[-1, 'singFam_2Story_NoBasement']= 0.6
df_agg.loc[-0.000000000001, 'singFam_2Story_NoBasement']= 0.6

# set curve for single family 3 story no basement
df_agg.loc[-1, 'singFam_3Story_NoBasement']= 0.0
df_agg.loc[-0.000000000001, 'singFam_3Story_NoBasement']= 0.0

# set curve for mobile home
df_agg.loc[-0.000000000001, 'mobileHome']= 0.75

df_agg = df_agg.sort_index()

# Weights for modelled runs

# Prep WSE/Attribute Data

In [9]:
#wse_file = 's3://pfra/RiskAssessment/{0}/Results/{1}/WSE_{0}_{1}_{2}.csv'.format(project, model_name, book)
breach_prob_file = 's3://pfra/RiskAssessment/{0}/BreachAnalysis/{0}_{1}_raw_prob_table.csv'.format(project, model_name)
weights_file = 's3://pfra/RiskAssessment/{0}/BreachAnalysis/{0}_{1}.xlsx'.format(project, model_name)

In [12]:
dfwse = pd.read_csv(wse_file, index_col='plus_code')
dfw = pd.read_excel(weights_file, sheet_name='Event_Weights', index_col=0 )[['Overall Weight']]
dfbp = pd.read_csv(breach_prob_file, sep='\t', index_col=0)
for col in dfbp.columns:
    dfbp.rename(columns={col:col.split('_')[1]}, inplace=True)

In [13]:
events_data_object = FluvialEvents(dfw, dfbp)
#allargs = [(p_code, events_data_object, df_agg, dfwse, dfw, dfbp, structure_cols) for p_code in dfwse.index]

In [14]:
#events_data_object.breach_locations

In [15]:
wse_results = [c for c in dfwse.columns if '_E' in c]
thin_df = dfwse[wse_results].copy()
thin_df.head()

,B01_E0029,B01_E0044,B01_E0061,B01_E0065,B01_E0068,B01_E0070,B01_E0072,B01_E0075,B01_E0078,B01_E0088,B01_E0098,B02_E0044,B02_E0065,B02_E0075,B02_E0078,B02_E0088,B07_E0065,B07_E0068,B07_E0070,B07_E0072,B07_E0075,B07_E0078,B07_E0088,B07_E0098,B08_E0072,B08_E0075,B08_E0078,B08_E0088,B08_E0098,B09_E0078,B09_E0088,B09_E0098,B10_E0070,B10_E0072,B10_E0075,B10_E0078,B10_E0088,B10_E0098,B11_E0070,B11_E0072,B11_E0075,B11_E0078,B11_E0088,B11_E0098,B14_E0001,B14_E0003,B14_E0008,B14_E0013,B14_E0018,B14_E0023,B14_E0029,B14_E0044,B14_E0061,B14_E0065,B14_E0068,B14_E0070,B14_E0072,B14_E0075,B14_E0078,B14_E0088,B14_E0098,B15_E0001,B15_E0003,B15_E0008,B15_E0013,B15_E0018,B15_E0023,B15_E0029,B15_E0044,B15_E0061,B15_E0065,B15_E0068,B15_E0070,B15_E0072,B15_E0075,B15_E0078,B15_E0088,B15_E0098,B16_E0001,B16_E0003,B16_E0008,B16_E0013,B16_E0018,B16_E0023,B16_E0029,B16_E0044,B16_E0061,B16_E0065,B16_E0068,B16_E0070,B16_E0072,B16_E0075,B16_E0078,B16_E0088,B16_E0098,B17_E0001,B17_E0003,B17_E0008,B17_E0013,B17_E0018,B17_E0023,B17_E0029,B17_E0044,B17_E0061,B17_E0065,B17_E0068,B17_E0070,B17_E0072,B17_E0075,B17_E0078,B17_E0088,B17_E0098,B20_E0001,B20_E0003,B20_E0008,B20_E0013,B20_E0018,B20_E0023,B20_E0029,B20_E0044,B20_E0061,B20_E0065,B20_E0068,B20_E0070,B20_E0072,B20_E0075,B20_E0078,B20_E0088,B20_E0098,B21_E0001,B21_E0003,B21_E0008,B21_E0013,B21_E0018,B21_E0023,B21_E0029,B21_E0044,B21_E0061,B21_E0065,B21_E0068,B21_E0070,B21_E0072,B21_E0075,B21_E0078,B21_E0088,B21_E0098,B22_E0001,B22_E0003,B22_E0008,B22_E0013,B22_E0018,B22_E0023,B22_E0029,B22_E0044,B22_E0061,B22_E0065,B22_E0068,B22_E0070,B22_E0072,B22_E0075,B22_E0078,B22_E0088,...,B97_E0098,B98_E0001,B98_E0003,B98_E0008,B98_E0013,B98_E0018,B98_E0023,B98_E0029,B98_E0044,B98_E0061,B98_E0065,B98_E0068,B98_E0070,B98_E0072,B98_E0075,B98_E0078,B98_E0088,B98_E0098,B99_E0001,B99_E0003,B99_E0008,B99_E0013,B99_E0018,B99_E0023,B99_E0029,B99_E0044,B99_E0061,B99_E0065,B99_E0068,B99_E0070,B99_E0072,B99_E0075,B99_E0078,B99_E0088,B99_E0098,NBR_E0001,NBR_E0003,NBR_E0008,NBR_E0013,NBR_E0018,NBR_E0023,NBR_E0029,NBR_E0044,NBR_E0061,NBR_E0065,NBR_E0068,NBR_E0070,NBR_E0072,NBR_E0075,NBR_E0078,NBR_E0088,NBR_E0098,Z01_E0001,Z01_E0003,Z01_E0008,Z01_E0013,Z01_E0018,Z01_E0023,Z01_E0029,Z01_E0044,Z01_E0061,Z01_E0065,Z01_E0068,Z01_E0070,Z01_E0072,Z01_E0075,Z01_E0078,Z01_E0088,Z01_E0098,Z02_E0001,Z02_E0003,Z02_E0008,Z02_E0013,Z02_E0018,Z02_E0023,Z02_E0029,Z02_E0044,Z02_E0061,Z02_E0065,Z02_E0068,Z02_E0075,Z02_E0078,Z02_E0098,Z03_E0001,Z03_E0003,Z03_E0008,Z03_E0013,Z03_E0018,Z03_E0023,Z03_E0029,Z03_E0044,Z03_E0061,Z03_E0065,Z03_E0068,Z03_E0070,Z03_E0072,Z03_E0075,Z03_E0078,Z03_E0088,Z03_E0098,Z04_E0044,Z04_E0061,Z04_E0065,Z04_E0068,Z04_E0070,Z04_E0072,Z04_E0075,Z04_E0078,Z04_E0088,Z04_E0098,Z05_E0001,Z05_E0003,Z05_E0008,Z05_E0013,Z05_E0018,Z05_E0023,Z05_E0029,Z05_E0044,Z05_E0061,Z05_E0065,Z05_E0068,Z05_E0070,Z05_E0072,Z05_E0075,Z05_E0078,Z05_E0088,Z05_E0098,Z06_E0001,Z06_E0003,Z06_E0008,Z06_E0013,Z06_E0018,Z06_E0023,Z06_E0029,Z06_E0044,Z06_E0061,Z06_E0065,Z06_E0068,Z06_E0070,Z06_E0072,Z06_E0075,Z06_E0078,Z06_E0088,Z06_E0098,Z24_E0008,Z24_E0013,Z24_E0018,Z24_E0023,Z24_E0029,Z24_E0044,Z24_E0061,Z24_E0065,Z24_E0068,Z24_E0070,Z24_E0072,Z24_E0075,Z24_E0078,Z24_E0088,Z24_E0098,Z26_E0088,Z26_E0098,Z27_E0098
plus_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
84CW252H+HF69933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [16]:
thin_df['Max'] = thin_df.max(axis=1)
non_zeros = thin_df[thin_df['Max'] > 0].index
print('Non-Zeros Points', len(non_zeros))

Non-Zeros Points 182935


In [22]:
zero_pts = [x for x in thin_df.index if x not in non_zeros]
print('Zeros Points', len(zero_pts))

Zeros Points 182917


In [27]:
zero_pts_results = [(pcode, 0) for pcode in zero_pts]

In [29]:
# Slice the full wsedf to select only plus codes with data to compute
compute_matrix = dfwse.loc[non_zeros].drop(columns='geom').copy()
compute_matrix.head()

,shp_nam,accntnum,location,bldg_ded,bldg_limit,cnt_ded,cnt_limit,state,postcode,country,lon,lat,bldg_value,cnt_value,constr_code,num_stories,year_built,foundationtype,basementfinishtype,first_floor_elev,base_flood_elev,elev_ft,damage_code,B01_E0029,B01_E0044,B01_E0061,B01_E0065,B01_E0068,B01_E0070,B01_E0072,B01_E0075,B01_E0078,B01_E0088,B01_E0098,B02_E0044,B02_E0065,B02_E0075,B02_E0078,B02_E0088,B07_E0065,B07_E0068,B07_E0070,B07_E0072,B07_E0075,B07_E0078,B07_E0088,B07_E0098,B08_E0072,B08_E0075,B08_E0078,B08_E0088,B08_E0098,B09_E0078,B09_E0088,B09_E0098,B10_E0070,B10_E0072,B10_E0075,B10_E0078,B10_E0088,B10_E0098,B11_E0070,B11_E0072,B11_E0075,B11_E0078,B11_E0088,B11_E0098,B14_E0001,B14_E0003,B14_E0008,B14_E0013,B14_E0018,B14_E0023,B14_E0029,B14_E0044,B14_E0061,B14_E0065,B14_E0068,B14_E0070,B14_E0072,B14_E0075,B14_E0078,B14_E0088,B14_E0098,B15_E0001,B15_E0003,B15_E0008,B15_E0013,B15_E0018,B15_E0023,B15_E0029,B15_E0044,B15_E0061,B15_E0065,B15_E0068,B15_E0070,B15_E0072,B15_E0075,B15_E0078,B15_E0088,B15_E0098,B16_E0001,B16_E0003,B16_E0008,B16_E0013,B16_E0018,B16_E0023,B16_E0029,B16_E0044,B16_E0061,B16_E0065,B16_E0068,B16_E0070,B16_E0072,B16_E0075,B16_E0078,B16_E0088,B16_E0098,B17_E0001,B17_E0003,B17_E0008,B17_E0013,B17_E0018,B17_E0023,B17_E0029,B17_E0044,B17_E0061,B17_E0065,B17_E0068,B17_E0070,B17_E0072,B17_E0075,B17_E0078,B17_E0088,B17_E0098,B20_E0001,B20_E0003,B20_E0008,B20_E0013,B20_E0018,B20_E0023,B20_E0029,B20_E0044,B20_E0061,B20_E0065,B20_E0068,B20_E0070,B20_E0072,B20_E0075,B20_E0078,B20_E0088,B20_E0098,B21_E0001,B21_E0003,B21_E0008,B21_E0013,B21_E0018,B21_E0023,B21_E0029,B21_E0044,B21_E0061,B21_E0065,...,B98_E0001,B98_E0003,B98_E0008,B98_E0013,B98_E0018,B98_E0023,B98_E0029,B98_E0044,B98_E0061,B98_E0065,B98_E0068,B98_E0070,B98_E0072,B98_E0075,B98_E0078,B98_E0088,B98_E0098,B99_E0001,B99_E0003,B99_E0008,B99_E0013,B99_E0018,B99_E0023,B99_E0029,B99_E0044,B99_E0061,B99_E0065,B99_E0068,B99_E0070,B99_E0072,B99_E0075,B99_E0078,B99_E0088,B99_E0098,NBR_E0001,NBR_E0003,NBR_E0008,NBR_E0013,NBR_E0018,NBR_E0023,NBR_E0029,NBR_E0044,NBR_E0061,NBR_E0065,NBR_E0068,NBR_E0070,NBR_E0072,NBR_E0075,NBR_E0078,NBR_E0088,NBR_E0098,Z01_E0001,Z01_E0003,Z01_E0008,Z01_E0013,Z01_E0018,Z01_E0023,Z01_E0029,Z01_E0044,Z01_E0061,Z01_E0065,Z01_E0068,Z01_E0070,Z01_E0072,Z01_E0075,Z01_E0078,Z01_E0088,Z01_E0098,Z02_E0001,Z02_E0003,Z02_E0008,Z02_E0013,Z02_E0018,Z02_E0023,Z02_E0029,Z02_E0044,Z02_E0061,Z02_E0065,Z02_E0068,Z02_E0075,Z02_E0078,Z02_E0098,Z03_E0001,Z03_E0003,Z03_E0008,Z03_E0013,Z03_E0018,Z03_E0023,Z03_E0029,Z03_E0044,Z03_E0061,Z03_E0065,Z03_E0068,Z03_E0070,Z03_E0072,Z03_E0075,Z03_E0078,Z03_E0088,Z03_E0098,Z04_E0044,Z04_E0061,Z04_E0065,Z04_E0068,Z04_E0070,Z04_E0072,Z04_E0075,Z04_E0078,Z04_E0088,Z04_E0098,Z05_E0001,Z05_E0003,Z05_E0008,Z05_E0013,Z05_E0018,Z05_E0023,Z05_E0029,Z05_E0044,Z05_E0061,Z05_E0065,Z05_E0068,Z05_E0070,Z05_E0072,Z05_E0075,Z05_E0078,Z05_E0088,Z05_E0098,Z06_E0001,Z06_E0003,Z06_E0008,Z06_E0013,Z06_E0018,Z06_E0023,Z06_E0029,Z06_E0044,Z06_E0061,Z06_E0065,Z06_E0068,Z06_E0070,Z06_E0072,Z06_E0075,Z06_E0078,Z06_E0088,Z06_E0098,Z24_E0008,Z24_E0013,Z24_E0018,Z24_E0023,Z24_E0029,Z24_E0044,Z24_E0061,Z24_E0065,Z24_E0068,Z24_E0070,Z24_E0072,Z24_E0075,Z24_E0078,Z24_E0088,Z24_E0098,Z26_E0088,Z26_E0098,Z27_E0098,GroundElev
plus_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
84CW295C+RG47G34,Sacramento_F01,CA07160777,CA07160777,1500,200000,1500,50000,CA,94511.0,US,-121.628675,38.009507,200000,50000,1,1,1990,8,0,1,9,0.000000,singFam_1Story_NoBasement,6.461713,6.794478,7.204110,7.331932,7.461139,7.508799,7.543277,7.583722,7.636795,7.704870,7.771173,6.758559,7.300850,7.569421,7.622909,7.688212,7.391128,7.502423,7.555858,7.583601,7.626585,7.671672,7.736001,7.801300,7.586311,7.629853

# Perform calculations

# Save Results

## Save Output AAL to csv

# Display Loss Statistics

# Anomalies

In [51]:
from importlib import reload
import risk_refactor
reload(risk_refactor)
from risk_refactor import *
events_data_object = FluvialEvents(dfw, dfbp)

In [54]:
step=100
results=[]
walltime=0
npoints =  compute_matrix.shape[0]

with Pool(int(cpu_count()/1)) as p:

    #for i in range(0, npoints, 100):
    for i in range(400, 500, step):
        test_slice = compute_matrix.iloc[i:i+step].copy()
        allargs = [(p_code, events_data_object, df_agg, test_slice.loc[p_code], structure_cols) for p_code in test_slice.index]

        st = time()
        if pluvial:
            slice_results = p.map(calc_pluv_aal_mp, allargs)
        else:
            slice_results = p.map(calc_fluv_aal_mp, allargs)

        results += slice_results
        wtime = time()-st
        walltime += wtime
        print('Progress = {}%'.format(round(100*(i/npoints)),2), '\tBatchtime =', round(wtime,2), 'seconds')
        #aal_results = {k:v for k,v in results}

    print(step, round(walltime/60, 2), 'minutes to process')

Progress = 0% 	Batchtime = 46.1 seconds
100 0.77 minutes to process


In [55]:
totime =round(walltime/60, 2)

tot = len(results)

trans= npoints/tot

(trans*totime)/60

23.476658333333333

In [50]:
#allargs[0]

# END 